**Лабараторна робота №2**

In [177]:
#Імпорт необхідних модулів
from datetime import datetime
import pandas as pd
import urllib
import os
print("Setup complete")

Setup complete


In [178]:
#Функція що повертає час в момент виклику
date_and_time_time = ''
def time_now():
    now = datetime.now()
    global date_and_time_time 
    date_and_time_time = now.strftime("%d%m%Y%H%M%S")
    print("Time now:",date_and_time_time)

In [179]:
#Функція оновлдення та заванатаження нових даних
def _download_data():
    #Оновл.ємо час
    time_now()
    
    #Масив з навзами областей України розташовані в алфавітному порядку відносно кирилиці
    index_ua = ['plashka', 22, 24, 23, 25, 3, 4, 8, 19, 20, 21, 9, 90, 10, 11, 12, 13, 14, 15, 16, 250, 17, 18, 6, 1, 2, 7, 5]
    
    #Перевіряємо чи папка з даними пуста та чи ця папка існує,
    #якщо ні, то відопвідно очищуємо, або створюємо нову папку
    if os.path.isdir("data"):
        print("Clean directory <data>...")
        for f in os.listdir('data'):
            os.remove("data/"+f)
        print("Complete")
    else:
        print("Create new directory <data>...")
        os.mkdir('data')
        print("Complete")

    #Початок завантаження даних
    print("Download...")
    
    for i in range(1,28):
        url='https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?provinceID={}&country=UKR&yearlyTag=Weekly&type=Mean&TagCropland=crop&year1=1982&year2=2023'.format(i)
        wp = urllib.request.urlopen(url)
        
        text = wp.read()
        out=open(f'data/NOAA_ID_{str(index_ua[i])}_{date_and_time_time}.csv','wb')

        #Очищення даних від різного сміття
        text = '\n'.join(text.decode().split("\n")[1:]).replace("<tt><pre>", '').replace("<br>", '').replace("</pre></tt>", '').replace(",", ';').encode()
        out.write(text)
        out.close()
    #replace("</pre></tt>", '')
    print("Complete")

_download_data()

Time now: 06102023042235
Clean directory <data>...
Complete
Download...
Complete


In [180]:
def get_dataframes(region_index):
    df = pd.read_csv(f'data/NOAA_ID_{region_index}_{date_and_time_time}.csv',
                     sep=';', index_col=False)
    df.columns = df.columns.str.replace(' ', '')
    df.dropna()
    return df

In [181]:
df=get_dataframes(5)
df.columns.values

array(['year', 'week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI'], dtype=object)

In [182]:
def take_data_year_week(df, year, week):
    return df[(df['year']==year)&(df['week']==week)]

take_data_year_week(df, 2000, 18)

,year,week,SMN,SMT,VCI,TCI,VHI
953,2000,18,0.377,289.99,83.84,51.33,67.58


In [167]:
def extremum(region, year):
    df=get_dataframes(region)
    year_df=df[df['year']==year]
    return [df[df['VHI']==min(year_df['VHI'])], df[df['VHI']==max(year_df['VHI'])]]

extremum(5, 2000)

[     year  week   SMN     SMT    VCI    TCI    VHI
 936  2000     1  0.04  261.43  15.25  38.99  27.12,
      year  week    SMN     SMT    VCI    TCI    VHI
 954  2000    19  0.408  291.21  83.05  52.47  67.76]

In [166]:
def extreme_drought(region, area_percent):
    df=get_dataframes(region)
    return df[ (df['VHI']<15) & (df['SMN']>area_percent/100) ]

extreme_drought(2, 2)

,year,week,SMN,SMT,VCI,TCI,VHI
653,1994,30,0.362,299.99,20.58,5.55,13.06
654,1994,31,0.338,298.92,13.89,9.20,11.54
655,1994,32,0.319,297.43,9.43,16.37,12.90


In [176]:
def moderate_drought(region, area_percent):
    df=get_dataframes(region)
    return df[ (df['VHI']>15) & (df['VHI']<35) & (df['SMN']>area_percent/100) ]

moderate_drought(2, 40)

,year,week,SMN,SMT,VCI,TCI,VHI
235,1986,28,0.414,295.85,41.55,27.20,34.37
544,1992,25,0.417,295.19,38.00,30.25,34.13
545,1992,26,0.419,295.74,38.82,24.40,31.61
546,1992,27,0.419,296.21,41.75,20.53,31.14
547,1992,28,0.416,296.60,43.03,18.00,30.52
548,1992,29,0.409,296.43,42.13,23.28,32.71
649,1994,26,0.448,297.31,56.54,12.50,34.52
650,1994,27,0.433,298.46,50.54,8.29,29.41
651,1994,28,0.411,299.40,40.40,5.48,22.94
707,1995,32,0.414,297.13,54.53,14.03,34.28
